In [1]:
import torch, os
from torch.autograd import Variable
import torch.nn as nn
import torch.onnx
import onnx 

from google.protobuf import json_format

dummy_input = Variable(torch.randn(2, 5))
model = torch.nn.Sequential(
          torch.nn.Linear(5, 4),
          torch.nn.Dropout(p=0.9),
          torch.nn.Sigmoid(),
          torch.nn.Linear(4, 3),
          torch.nn.Dropout(p=0.9),
          torch.nn.ReLU(),
          torch.nn.Linear(3, 2),
          torch.nn.Softmax(dim=1),
        )
torch_proto_path = os.path.join("/tmp", "torch_model.proto")
torch.onnx.export(model, dummy_input, torch_proto_path, verbose=True)

graph(%1 : Float(2, 5)
      %2 : Float(4, 5)
      %3 : Float(4)
      %4 : Float(3, 4)
      %5 : Float(3)
      %6 : Float(2, 3)
      %7 : Float(2)) {
  %10 : Float(2, 4) = Gemm[alpha=1, beta=1, broadcast=1, transB=1](%1, %2, %3), uses = [%11.i0];
  %12 : Float(2, 4), %13 : UNKNOWN_TYPE = Dropout[is_test=1, ratio=0.9](%10), uses = [[%14.i0], []];
  %14 : Float(2, 4) = Sigmoid(%12), uses = [%17.i0];
  %17 : Float(2, 3) = Gemm[alpha=1, beta=1, broadcast=1, transB=1](%14, %4, %5), uses = [%18.i0];
  %19 : Float(2, 3), %20 : UNKNOWN_TYPE = Dropout[is_test=1, ratio=0.9](%17), uses = [[%21.i0], []];
  %21 : Float(2, 3) = Relu(%19), uses = [%24.i0];
  %24 : Float(2, 2) = Gemm[alpha=1, beta=1, broadcast=1, transB=1](%21, %6, %7), uses = [%25.i0];
  %25 : Float(2, 2) = Softmax[axis=1](%24), uses = [%0.i0];
  return (%25);
}



In [2]:
model = onnx.load(torch_proto_path)
onnx.checker.check_model(model)
onnx.helper.strip_doc_string(model) # Cleaning
print(model)
####### onnx.load under the hood calls
# with open("model.proto", "rb") as f:
#     s = f.read()
# model = onnx.onnx_pb.ModelProto()
# model.ParseFromString(s)


ir_version: 2
producer_name: "pytorch"
producer_version: "0.3"
graph {
  node {
    input: "1"
    input: "2"
    input: "3"
    output: "10"
    op_type: "Gemm"
    attribute {
      name: "alpha"
      f: 1.0
      type: FLOAT
    }
    attribute {
      name: "beta"
      f: 1.0
      type: FLOAT
    }
    attribute {
      name: "broadcast"
      i: 1
      type: INT
    }
    attribute {
      name: "transB"
      i: 1
      type: INT
    }
  }
  node {
    input: "10"
    output: "12"
    output: "13"
    op_type: "Dropout"
    attribute {
      name: "is_test"
      i: 1
      type: INT
    }
    attribute {
      name: "ratio"
      f: 0.8999999761581421
      type: FLOAT
    }
  }
  node {
    input: "12"
    output: "14"
    op_type: "Sigmoid"
  }
  node {
    input: "14"
    input: "4"
    input: "5"
    output: "17"
    op_type: "Gemm"
    attribute {
      name: "alpha"
      f: 1.0
      type: FLOAT
    }
    attribute {
      name: "beta"
      f: 1.0
      type: FLOAT
 

In [3]:
import syft
import syft.nn
import syft.onnx
import imp
imp.reload(syft)
imp.reload(syft.nn)

import numpy as np
from syft import FloatTensor

dummy_input = FloatTensor([[1,2,3,4,5],[5,4,3,2,1]], autograd=True)
model = syft.nn.Sequential([
    syft.nn.Linear(5,3),
    syft.nn.Dropout(rate=0.9),
    syft.nn.Sigmoid(),
    syft.nn.Linear(3, 2),
    syft.nn.Dropout(rate=0.9),
    syft.nn.ReLU(),
    syft.nn.Linear(2, 1),
    syft.nn.Softmax(dim=1)
])
syft_proto_path = os.path.join("/tmp", "syft_model.proto")

/Users/morgangiraud/.virtualenvs/openmined/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [4]:
syft_model_as_json = syft.onnx.export_as_json(model, dummy_input, verbose=True)
model_from_syft = onnx.onnx_pb.ModelProto()
json_format.Parse(syft_model_as_json, model_from_syft)
onnx.checker.check_model(model_from_syft)
print(model_from_syft)

ValidationError: Attribute should contain one and only one value field.

==> Context: Bad node spec: input: "1" input: "2" output: "5" name: "linear" op_type: "Gemm" attribute { name: "alpha" f: 1 type: FLOAT } attribute { name: "beta" f: 1 type: FLOAT } attribute { name: "broadcast" i: 1 type: INT } attribute { name: "transB" type: INT }

In [5]:
fullpath = syft.onnx.export(model, dummy_input, syft_proto_path, verbose=True)
loaded_model = onnx.load(fullpath)
onnx.checker.check_model(model)
onnx.helper.strip_doc_string(loaded_model) # Cleaning
print(loaded_model)

ir_version: 2
producer_name: "openmined"
producer_version: "0.0.2"
domain: "org.openmined"
graph {
  node {
    input: "33"
    input: "34"
    output: "37"
    name: "linear"
    op_type: "Gemm"
    attribute {
      name: "alpha"
      f: 1.0
      type: FLOAT
    }
    attribute {
      name: "beta"
      f: 1.0
      type: FLOAT
    }
    attribute {
      name: "broadcast"
      i: 1
      type: INT
    }
    attribute {
      name: "transB"
      type: INT
    }
  }
  node {
    input: "37"
    output: "40"
    output: "38"
    name: "dropout"
    op_type: "Dropout"
    attribute {
      name: "ratio"
      f: 0.8999999761581421
      type: FLOAT
    }
    attribute {
      name: "is_test"
      i: 1
      type: INT
    }
  }
  node {
    input: "40"
    output: "41"
    op_type: "Sigmoid"
  }
  node {
    input: "41"
    input: "35"
    output: "42"
    name: "linear"
    op_type: "Gemm"
    attribute {
      name: "alpha"
      f: 1.0
      type: FLOAT
    }
    attribute {
   